
---

# 🧩 Pydantic Types

### 🎯 Intent

* Use Pydantic’s built-in types (e.g., Email, URL, UUID, Payment Card) to validate real-world data out of the box — no need to reimplement validators.
* Pydantic provides a wide range of type helpers that ensure your data is valid and type-safe.

---

### 🧩 Core Components

1. **📧 EmailStr**

   * Ensures valid email format.
   * Rejects invalid addresses automatically.

2. **🔗 AnyUrl & HttpUrl**

   * `AnyUrl` → accepts any valid URL.
   * `HttpUrl` → restricted to HTTP/HTTPS only.

3. **🆔 UUID**

   * Parses and validates UUID strings.
   * Stored as a `uuid.UUID` object internally.

4. **💳 PaymentCardNumber**

   * Validates credit/debit card numbers.
   * Includes length + checksum checks.

5. **📅 Date & Time Types**

   * `PastDate` → must be before today.
   * `FutureDate` → must be after today.
   * `AwareDatetime` → requires timezone info.

6. **🌐 IP Addresses**

   * `IPvAnyAddress`, `IPvAnyInterface`, `IPvAnyNetwork`.
   * Ensures valid IPv4/IPv6 addresses and networks.

7. **🎨 Color**

   * Accepts named colors (`"red"`), hex (`"#ff0000"`), RGB/RGBA formats.

8. **🔒 SecretStr / SecretBytes**

   * Stores sensitive values (e.g., passwords, tokens).
   * Automatically masks when printed/logged.

9. **💰 Decimal & Constrained Numbers**

   * `Decimal` → exact precision for money.
   * Constrained: `conint`, `confloat`, `PositiveInt`, `NonNegativeFloat`.

10. **📜 Constrained Strings**

* `constr(min_length=..., max_length=..., pattern=...)`.
* Useful for usernames, IDs, regex-validated fields.

11. **📦 File & Directory Paths**

* `FilePath`, `DirectoryPath` → must exist on disk.
* `Path` → any valid filesystem path.

12. **📞 Miscellaneous**

* `Json` → validates JSON strings.
* `Base64Bytes` → ensures valid base64 encoding.
* `PaymentCardBrand` → detects card brand from number.

---

In [7]:
from pydantic import (
    BaseModel, EmailStr, AnyUrl, HttpUrl, UUID4,
    PastDate, FutureDate, AwareDatetime,
    SecretStr, conint, constr, FilePath, DirectoryPath,
    Json, Base64Bytes, IPvAnyAddress
)
from pydantic_extra_types.color import Color
from pydantic_extra_types.payment import PaymentCardNumber
from decimal import Decimal
from pathlib import Path
from datetime import date, datetime, timedelta, timezone
import uuid, base64, tempfile

class DemoModel(BaseModel):
    email: EmailStr
    website: AnyUrl
    homepage: HttpUrl
    user_id: UUID4
    card_number: PaymentCardNumber
    birthday: PastDate
    next_meeting: FutureDate
    login_time: AwareDatetime
    ip: IPvAnyAddress
    favorite_color: Color
    password: SecretStr
    balance: Decimal
    age: conint(ge=18, le=120)
    username: constr(min_length=3, max_length=20, pattern=r"^[a-zA-Z0-9_]+$")
    config_file: FilePath         # must be an existing file
    data_dir: DirectoryPath       # must be an existing directory
    metadata: Json
    file_blob: Base64Bytes

# --- create a guaranteed-existing temp file for FilePath ---
tmp = tempfile.NamedTemporaryFile(delete=False)
tmp_path = Path(tmp.name)
tmp.close()

demo = DemoModel(
    email="test@example.com",
    website="https://pydantic.dev",
    homepage="https://fastapi.tiangolo.com",
    user_id=uuid.uuid4(),
    card_number="4111111111111111",
    birthday=date(2000, 1, 1),
    next_meeting=date.today() + timedelta(days=10),
    login_time=datetime.now(timezone.utc),
    ip="127.0.0.1",
    favorite_color="red",
    password="supersecret",
    balance=Decimal("1234.56"),
    age=25,
    username="Mukesh_Y",
    config_file=tmp_path,          # ✅ now a real file
    data_dir=Path("."),            # ✅ current directory exists
    metadata='{"key": "value"}',
    file_blob=base64.b64encode(b"hello world")
)

print(demo.model_dump(exclude={"password"}))
print("Password stored as:", demo.password)


{'email': 'test@example.com', 'website': AnyUrl('https://pydantic.dev/'), 'homepage': HttpUrl('https://fastapi.tiangolo.com/'), 'user_id': UUID('4be0cb40-7a67-4635-b2af-02e0bd988119'), 'card_number': '4111111111111111', 'birthday': datetime.date(2000, 1, 1), 'next_meeting': datetime.date(2025, 9, 19), 'login_time': datetime.datetime(2025, 9, 9, 4, 42, 12, 788036, tzinfo=datetime.timezone.utc), 'ip': IPv4Address('127.0.0.1'), 'favorite_color': Color('red', rgb=(255, 0, 0)), 'balance': Decimal('1234.56'), 'age': 25, 'username': 'Mukesh_Y', 'config_file': PosixPath('/var/folders/br/18cx1rhn5j5d94v9wk8sk38m0000gn/T/tmpu5h_2npp'), 'data_dir': PosixPath('.'), 'metadata': {'key': 'value'}, 'file_blob': b'aGVsbG8gd29ybGQ='}
Password stored as: **********
